In [4]:
#from sklearn.model_selection import train_test_split
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import twitter_samples, stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk import FreqDist, classify, NaiveBayesClassifier

In [5]:
import re, string, random

In [6]:
import math
import random
from collections import defaultdict
from pprint import pprint
from collections import Counter
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import re
import string


import numpy as np
import pandas as pd


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
data = pd.read_excel(r'India Product Review 2021.xlsx',header=2,nrows=10000)
data.head()

,CHANNEL SKU,PRODUCT NAME,CHANNEL,REVIEW DATE,CUSTOMER NAME,RATING BY CUSTOMER,REVIEW TYPE,VERIFIED PURCHASE,REVIEW TITLE,REVIEW TEXT
0,DPRFEMZUADBSUPDU,Huggies New Dry Tape Diapers - L 30 Pieces,Flipkart,2021-04-18,sajith KUMAR,4,POSITIVE,True,Good choice,Nice product.\nValue for money
1,DPRFMTQUFNC5HSQX,Huggies New Dry Tape Diapers -Combo Pack - L 1...,Flipkart,2021-04-18,sajith KUMAR,4,POSITIVE,True,Good choice,Nice product.\nValue for money
2,DPRFMKHZZJKZ3KXG,Huggies Dry Pants Diapers - S 10 Pieces,Flipkart,2021-04-18,Jhily Pattnaik,5,POSITIVE,True,Just wow!,Nice
3,DPRFMKHZZJKZ3KXG,Huggies Dry Pants Diapers - S 10 Pieces,Flipkart,2021-04-18,Flipkart Customer,5,POSITIVE,True,Brilliant,Very Comfortable for kids. Fits perfectly on 8...
4,DPRFMKHZZJKZ3KXG,Huggies Dry Pants Diapers - S 10 Pieces,Flipkart,2021-04-18,Flipkart Customer,5,POSITIVE,True,Great product,Nice quality


In [8]:
df=data[['REVIEW TEXT','REVIEW TYPE']]
df.head()

,REVIEW TEXT,REVIEW TYPE
0,Nice product.\nValue for money,POSITIVE
1,Nice product.\nValue for money,POSITIVE
2,Nice,POSITIVE
3,Very Comfortable for kids. Fits perfectly on 8...,POSITIVE
4,Nice quality,POSITIVE


In [9]:
df=df.dropna()
df['labels'] = np.where(df['REVIEW TYPE'] == "POSITIVE", 1, 0)
df.head()
df.tail(10)

,REVIEW TEXT,REVIEW TYPE,labels
9990,Good product,POSITIVE,1
9991,Average,NEGATIVE,0
9992,Very nice product and good quality,POSITIVE,1
9993,Too odors making me worried about the chemical...,NEGATIVE,0
9994,Goof,POSITIVE,1
9995,Value for money n very long lasting,POSITIVE,1
9996,Very good,POSITIVE,1
9997,Good,POSITIVE,1
9998,Good,POSITIVE,1
9999,Very Bad experience,NEGATIVE,0


In [10]:
stop = set(stopwords.words('english'))

In [11]:
def clean_document(doco):
    punctuation = string.punctuation
    punc_replace = ''.join([' ' for s in punctuation])
    doco_link_clean = re.sub(r'http\S+', '', doco)
    doco_clean_and = re.sub(r'&\S+', '', doco_link_clean)
    doco_clean_at = re.sub(r'@\S+', '', doco_clean_and)
    doco_clean = doco_clean_at.replace('-', ' ')
    doco_alphas = re.sub(r'\W +', ' ', doco_clean)
    trans_table = str.maketrans(punctuation, punc_replace)
    doco_clean = ' '.join([word.translate(trans_table) for word in doco_alphas.split(' ')])
    doco_clean = doco_clean.split(' ')
    p = re.compile(r'\s*\b(?=[a-z\d]*([a-z\d])\1{3}|\d+\b)[a-z\d]+', re.IGNORECASE)
    doco_clean = ([p.sub("", x).strip() for x in doco_clean])
    doco_clean = [word.lower() for word in doco_clean if len(word) > 2]
    doco_clean = ([i for i in doco_clean if i not in stop])
#     doco_clean = [spell(word) for word in doco_clean]
#     p = re.compile(r'\s*\b(?=[a-z\d]*([a-z\d])\1{3}|\d+\b)[a-z\d]+', re.IGNORECASE)
    doco_clean = ([p.sub("", x).strip() for x in doco_clean])
#     doco_clean = ([spell(k) for k in doco_clean])
    return doco_clean

In [12]:
review_clean = [clean_document(doc) for doc in df['REVIEW TEXT']]
sentences = [' '.join(r) for r in review_clean]

In [13]:
df['cleantext']=sentences
df.head()

,REVIEW TEXT,REVIEW TYPE,labels,cleantext
0,Nice product.\nValue for money,POSITIVE,1,nice product value money
1,Nice product.\nValue for money,POSITIVE,1,nice product value money
2,Nice,POSITIVE,1,nice
3,Very Comfortable for kids. Fits perfectly on 8...,POSITIVE,1,comfortable kids fits perfectly months kid
4,Nice quality,POSITIVE,1,nice quality


In [14]:
train_positive_sentiment = df[df.labels == 1]
positive_words = ' '.join(train_positive_sentiment['cleantext'])
positive_words=list(positive_words.split(' '))
print("Top words in Positive Sentiment")
#print(positive_words)

Top words in Positive Sentiment


In [15]:
train_negative_sentiment = df[df.labels == 0]
negative_words = ' '.join(train_positive_sentiment['cleantext'])
negative_words = negative_words.split()
#print("Top words in Negative Sentiment")
#train_negative_sentiment.tail(20)

In [16]:
def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

In [17]:
positive_tokens_for_model = get_tweets_for_model(positive_words)
negative_tokens_for_model = get_tweets_for_model(negative_words)

In [18]:
positive_dataset = [(tweet_dict, "POSITIVE")
                         for tweet_dict in positive_tokens_for_model]

negative_dataset = [(tweet_dict, "NEGATIVE")
                         for tweet_dict in negative_tokens_for_model]

dataset = positive_dataset + negative_dataset

In [19]:
random.shuffle(dataset)

In [20]:
print(len(dataset))

55709


In [21]:
classifier = NaiveBayesClassifier.train(dataset)

In [22]:
import pickle

In [23]:
filename='model.pkl'
pickle.dump(classifier, open(filename,'wb'))

In [24]:
test_data=pd.read_excel(r'India Product Review 2021.xlsx',header=2,skiprows=[i for i in range(3,20000)],skipfooter=70000)

In [25]:
print(test_data)

           CHANNEL SKU                                    PRODUCT NAME  \
0     DPRFCMVAYEKEGENU      Huggies Wonder Pants Diapers - S 86 Pieces   
1     DPRF8CKFGQBB9ZM7      Huggies Wonder Pants diapers - S 20 Pieces   
2     DPRF8CKFGQBB9ZM7      Huggies Wonder Pants diapers - S 20 Pieces   
3     DPRFCMVAYEKEGENU      Huggies Wonder Pants Diapers - S 86 Pieces   
4     DPRFCMVAYEKEGENU      Huggies Wonder Pants Diapers - S 86 Pieces   
...                ...                                             ...   
8234  DPRFZ2W3ZACGDK44   Huggies Nature care pant Diaper - M 22 Pieces   
8235  DPRFZ2W3ZACGDK44   Huggies Nature care pant Diaper - M 22 Pieces   
8236  DPRFZ2W3ZACGDK44   Huggies Nature care pant Diaper - M 22 Pieces   
8237  DPRFZ2W2U4HVGG7G  Huggies Nature care pant Diaper - S 164 Pieces   
8238  DPRFZ2W2U4HVGG7G  Huggies Nature care pant Diaper - S 164 Pieces   

       CHANNEL REVIEW DATE        CUSTOMER NAME  RATING BY CUSTOMER  \
0     Flipkart  2021-03-18  Sivagami Mut

In [26]:
test_data.head()

,CHANNEL SKU,PRODUCT NAME,CHANNEL,REVIEW DATE,CUSTOMER NAME,RATING BY CUSTOMER,REVIEW TYPE,VERIFIED PURCHASE,REVIEW TITLE,REVIEW TEXT
0,DPRFCMVAYEKEGENU,Huggies Wonder Pants Diapers - S 86 Pieces,Flipkart,2021-03-18,Sivagami Muthukumar,5,POSITIVE,True,Simply awesome,Fantastic
1,DPRF8CKFGQBB9ZM7,Huggies Wonder Pants diapers - S 20 Pieces,Flipkart,2021-03-18,Flipkart Customer,5,POSITIVE,True,Mind-blowing purchase,Super
2,DPRF8CKFGQBB9ZM7,Huggies Wonder Pants diapers - S 20 Pieces,Flipkart,2021-03-18,Flipkart Customer,5,POSITIVE,True,Wonderful,Nice product \n.. super good
3,DPRFCMVAYEKEGENU,Huggies Wonder Pants Diapers - S 86 Pieces,Flipkart,2021-03-18,Flipkart Customer,5,POSITIVE,True,Mind-blowing purchase,Very very nice
4,DPRFCMVAYEKEGENU,Huggies Wonder Pants Diapers - S 86 Pieces,Flipkart,2021-03-18,Flipkart Customer,1,NEGATIVE,True,Terrible product,Very bad quality diapers and I ordered 42 uni...


In [51]:
def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

In [52]:
test_data["Model_Rating"]=["" for i in range(len(test_data))]
test_data["tokens"]=["" for i in range(len(test_data))]

In [53]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [57]:
review_clean = [clean_document(str(doc)) for doc in test_data['REVIEW TEXT']]
sentences = [' '.join(r) for r in review_clean]
test_data['tokens']=sentences

In [58]:
#l=[]
for index,row in test_data.iterrows():
    custom_tweet = str(row["REVIEW TEXT"])
    #print(custom_tweet)
    custom_tokens = remove_noise(word_tokenize(custom_tweet))
    #l.append(classifier.classify(dict([token, True] for token in custom_tokens)))
    test_data.loc[index, "Model_Rating"] = classifier.classify(dict([token, True] for token in custom_tokens))
#print(l)

In [59]:
test_data.head()

,CHANNEL SKU,PRODUCT NAME,CHANNEL,REVIEW DATE,CUSTOMER NAME,RATING BY CUSTOMER,REVIEW TYPE,VERIFIED PURCHASE,REVIEW TITLE,REVIEW TEXT,Model_Rating,tokens
0,DPRFCMVAYEKEGENU,Huggies Wonder Pants Diapers - S 86 Pieces,Flipkart,2021-03-18,Sivagami Muthukumar,5,POSITIVE,True,Simply awesome,Fantastic,POSITIVE,fantastic
1,DPRF8CKFGQBB9ZM7,Huggies Wonder Pants diapers - S 20 Pieces,Flipkart,2021-03-18,Flipkart Customer,5,POSITIVE,True,Mind-blowing purchase,Super,POSITIVE,super
2,DPRF8CKFGQBB9ZM7,Huggies Wonder Pants diapers - S 20 Pieces,Flipkart,2021-03-18,Flipkart Customer,5,POSITIVE,True,Wonderful,Nice product \n.. super good,POSITIVE,nice product super good
3,DPRFCMVAYEKEGENU,Huggies Wonder Pants Diapers - S 86 Pieces,Flipkart,2021-03-18,Flipkart Customer,5,POSITIVE,True,Mind-blowing purchase,Very very nice,POSITIVE,nice
4,DPRFCMVAYEKEGENU,Huggies Wonder Pants Diapers - S 86 Pieces,Flipkart,2021-03-18,Flipkart Customer,1,NEGATIVE,True,Terrible product,Very bad quality diapers and I ordered 42 uni...,NEGATIVE,bad quality diapers ordered unit unit


In [61]:
test_data.to_excel('Results with amazon data_3.xlsx',index=False)